In [1]:
from sklearn.svm import SVC
import pandas as pd
import numpy as np
from sklearn.neural_network import MLPClassifier

In [2]:
import keras
import pandas as pd
import numpy as np
import random
import math
from keras.optimizers import Adam
from keras.callbacks import TensorBoard, LearningRateScheduler, EarlyStopping, ReduceLROnPlateau
from keras.layers import Dense, Dropout, Input, Activation, Flatten, LeakyReLU, Add, GaussianNoise
from keras.layers import LSTM, Embedding, Conv1D, GlobalAveragePooling1D, ZeroPadding1D, MaxPooling1D, GRU, UpSampling1D
from keras.layers import Conv2D, GlobalAveragePooling2D, ZeroPadding2D, MaxPooling2D, UpSampling2D
from keras.layers.merge import concatenate
from keras.models import Model, Sequential, load_model, model_from_json
from keras.layers.normalization import BatchNormalization
from keras.backend import clear_session
from keras import regularizers
from keras.utils import plot_model
from IPython.display import Image
from keras import backend as K
import tensorflow as tf

import csv
import os
from keras.utils import to_categorical
import matplotlib.pyplot as plt

Using TensorFlow backend.


In [4]:
Distance = 'cosine'
Train = pd.read_csv(f'.\data\Train_v2_{Distance}.csv', index_col = 0)
Label = Train['Loan_Status']
Train = Train.drop('Loan_Status',axis = 1)
Test = pd.read_csv(f'.\data\Test_v2_{Distance}.csv', index_col = 0)

In [5]:
print(Train.shape)
Train

(614, 16)


,Gender,Married,Education,Self_Employed,ApplicantIncome,CoapplicantIncome,LoanAmount,Loan_Amount_Term,Credit_History,Dependents_0,Dependents_1,Dependents_2,Dependents_3+,Property_Area_Rural,Property_Area_Semiurban,Property_Area_Urban
Loan_ID,,,,,,,,,,,,,,,,
LP001002,0.0,0.0,1.0,0.0,0.072210,0.000000,0.214286,0.750,1.0,1.0,0.0,0.0,0.0,0,0,1
LP001003,0.0,1.0,1.0,0.0,0.056580,0.036192,0.182857,0.750,1.0,0.0,1.0,0.0,0.0,1,0,0
LP001005,0.0,1.0,1.0,1.0,0.037037,0.000000,0.094286,0.750,1.0,1.0,0.0,0.0,0.0,0,0,1
LP001006,0.0,1.0,0.0,0.0,0.031889,0.056592,0.171429,0.750,1.0,1.0,0.0,0.0,0.0,0,0,1
LP001008,0.0,0.0,1.0,0.0,0.074074,0.000000,0.201429,0.750,1.0,1.0,0.0,0.0,0.0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
LP002978,1.0,0.0,1.0,0.0,0.035802,0.000000,0.101429,0.750,1.0,1.0,0.0,0.0,0.0,1,0,0
LP002979,0.0,1.0,1.0,0.0,0.050691,0.000000,0.057143,0.375,1.0,0.0,0.0,0.0,1.0,1,0,0
LP002983,0.0,1.0,1.0,0.0,0.099654,0.005760,0.361429,0.750,1.0,0.0,1.0,0.0,0.0,0,0,1


In [6]:
print(Test.shape)
Test.describe()

(367, 16)


,Gender,Married,Education,Self_Employed,ApplicantIncome,CoapplicantIncome,LoanAmount,Loan_Amount_Term,Credit_History,Dependents_0,Dependents_1,Dependents_2,Dependents_3+,Property_Area_Rural,Property_Area_Semiurban,Property_Area_Urban
count,367.000000,367.000000,367.000000,367.000000,367.000000,367.000000,367.000000,367.000000,367.000000,367.000000,367.000000,367.000000,367.000000,367.000000,367.000000,367.000000
mean,0.198910,0.634877,0.771117,0.108992,0.059328,0.037670,0.194270,0.713193,0.833787,0.561308,0.160763,0.166213,0.111717,0.302452,0.316076,0.381471
std,0.399725,0.482122,0.420687,0.312054,0.060626,0.056021,0.087414,0.135850,0.372780,0.496905,0.367814,0.372780,0.315448,0.459947,0.465578,0.486411
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.040000,0.012500,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,0.000000,0.000000,1.000000,0.000000,0.035358,0.000000,0.142857,0.750000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,0.000000,1.000000,1.000000,0.000000,0.046741,0.024600,0.178571,0.750000,1.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
75%,0.000000,1.000000,1.000000,0.000000,0.062469,0.058332,0.225714,0.750000,1.000000,1.000000,0.000000,0.000000,0.000000,1.000000,1.000000,1.000000
max,1.000000,1.000000,1.000000,1.000000,0.895420,0.575995,0.785714,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000


# SVM

In [7]:
Gamma = 0.5
C = 1
clf = SVC(gamma= Gamma, C= C, kernel='linear', class_weight='balanced')
clf.fit(Train.values, Label.values) 
OUT = clf.predict(Test.values)
print(OUT)
print(len(OUT))
OUT = [str(i) for i in OUT]

[1 1 1 1 1 1 1 0 1 1 1 1 1 0 1 1 1 1 1 1 1 1 1 1 1 0 1 1 1 1 1 1 1 1 1 0 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 0 1 1 0 1 1 1 1 0 1 1 0 0 1 0 1 1 1 1
 1 1 1 1 1 1 0 1 0 1 0 1 1 1 1 1 1 1 1 1 0 1 1 1 1 1 1 0 1 1 1 1 0 1 1 1 1
 1 1 1 1 1 1 0 0 0 1 1 1 0 0 1 0 1 1 1 1 1 1 1 1 1 1 1 1 1 0 1 0 1 1 1 1 0
 1 1 1 1 1 0 1 1 1 1 1 1 1 0 1 1 1 0 0 1 0 1 1 1 1 0 0 1 1 1 1 1 1 1 1 1 1
 0 1 1 1 1 1 1 0 0 1 1 0 1 0 1 1 1 1 1 1 1 1 1 1 1 1 0 1 1 1 1 1 1 1 1 1 1
 1 1 0 1 1 1 1 0 1 1 1 1 1 0 0 1 1 1 1 0 1 0 1 0 1 1 1 1 0 1 1 1 1 0 1 1 1
 1 1 1 1 1 1 1 0 1 0 1 1 1 1 0 0 1 1 1 0 1 1 1 1 1 1 1 1 1 1 1 1 1 1 0 1 1
 1 1 1 1 1 0 1 1 1 1 1 1 1 1 1 0 1 1 1 1 1 0 1 1 1 1 1 1 1 0 1 1 1 1 1 1 1
 1 1 1 1 1 1 0 1 1 1 1 1 1 0 1 1 1 1 0 1 1 0 1 1 1 1 1 1 1 1 1 1 1 1]
367


In [9]:
ANS = pd.DataFrame({'Loan_ID':Test.index, 'Loan_Status':OUT}).replace({'1':'Y', '0':'N'})

In [10]:
ANS.to_csv(f'.\data\SVM_{Distance}_C_'+str(C)+'_Gamma_'+str(Gamma)+'_V2.csv',index = 0)

In [47]:
ANS

,Loan_ID,Loan_Status
0,LP001015,Y
1,LP001022,Y
2,LP001031,Y
3,LP001035,Y
4,LP001051,Y
...,...,...
362,LP002971,N
363,LP002975,Y
364,LP002980,Y
365,LP002986,Y


# MLP

In [126]:
L = 5
hidden = 1
clf = MLPClassifier(solver='adam', alpha=1e-5,hidden_layer_sizes=(L,hidden),max_iter=1000, random_state=1,learning_rate = 'adaptive')
clf.fit(Train.values, Label.values) 
OUT = clf.predict(Test.values)
print(OUT)
print(len(OUT))
OUT = [str(i) for i in OUT]

[1 1 1 1 1 1 1 0 1 1 1 1 1 0 1 1 1 1 1 1 1 1 1 1 1 0 1 1 1 1 1 1 1 1 1 0 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 0 1 1 0 1 1 1 1 0 1 1 0 0 1 0 1 1 1 1
 1 1 1 1 1 1 0 1 0 1 1 1 1 1 1 1 1 1 1 1 0 1 1 1 1 1 1 0 1 1 1 1 0 1 1 1 1
 1 1 1 1 0 1 0 0 0 1 1 1 0 0 1 0 1 1 1 1 1 1 1 1 1 1 1 1 1 0 1 0 1 1 1 1 0
 1 1 1 1 1 0 1 1 1 1 1 1 1 0 1 1 1 0 0 1 0 1 1 1 1 0 0 1 1 1 1 1 1 1 1 1 1
 0 1 1 1 1 1 1 0 0 1 1 0 1 0 1 1 1 1 1 1 1 1 1 1 0 1 0 1 1 1 1 1 1 1 1 1 1
 1 1 0 1 1 1 1 0 1 1 1 1 1 0 0 1 1 1 1 0 1 0 1 0 1 1 1 1 0 1 1 1 1 0 1 1 1
 1 1 1 1 1 1 1 0 0 0 1 1 1 1 0 0 1 1 1 0 1 1 1 1 1 1 1 1 1 1 1 1 1 1 0 1 1
 1 1 1 1 1 0 1 1 1 1 1 1 1 1 1 0 1 1 1 1 1 0 1 1 1 1 1 1 1 0 1 1 1 1 1 1 1
 1 1 1 1 1 1 0 1 1 1 1 1 1 0 1 1 1 1 0 1 1 0 1 1 1 1 1 1 1 1 1 1 1 1]
367


In [127]:
ANS = pd.DataFrame({'Loan_ID':Test.index, 'Loan_Status':OUT}).replace({'1':'Y', '0':'N'})

In [128]:
ANS.to_csv('.\data\MLP_L_'+str(L)+ '_hidden_'+ str(hidden) +'_V2.csv',index = 0)

# using Keras

In [11]:
#compute imbalancing
SUM_1 = np.sum(Label)
class_weights = [SUM_1/Label.shape[0], (Label.shape[0]-SUM_1)/Label.shape[0]]
print(class_weights)

[0.6872964169381107, 0.3127035830618892]


In [10]:
from sklearn.utils import class_weight
class_weights = class_weight.compute_class_weight('balanced',np.unique(Label.values),Label.values)
print(class_weights)

[1.59895833 0.72748815]


In [36]:
input_size = 4
output_size = 1
L2 = 7e-4
L_rate = 0.01
OUT = []
# for i in range(10):
INPUT =Input(shape = (16,))
x = Dense(input_size, kernel_regularizer = regularizers.l2( L2))(INPUT)
x = Activation('relu')(x)
x = Dense(output_size, kernel_regularizer = regularizers.l2( L2))(x)
out = Activation('sigmoid')(x)

model = Model(inputs=INPUT, outputs=out)
model.summary()
MY_LR = keras.callbacks.ReduceLROnPlateau(monitor='acc', factor=0.5, patience=40, mode='auto', min_delta=0.0001)
MY_ES = keras.callbacks.EarlyStopping(monitor='val_acc', min_delta=0, patience=100,\
                                      verbose=2, mode='auto', restore_best_weights=True)
adam=Adam(lr=L_rate, epsilon=1e-8, decay=0)
model.compile(loss='binary_crossentropy',optimizer=adam,metrics=['accuracy'])
model.fit(Train.values, Label.values, batch_size=256, epochs=1000, verbose=2,\
          class_weight = class_weights,validation_split = 0.2, callbacks = [MY_LR,MY_ES])
OUT = model.predict(Test.values)

print(OUT)
# print(len(OUT))
#OUT = [str(i) for i in OUT]

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_9 (InputLayer)         (None, 16)                0         
_________________________________________________________________
dense_17 (Dense)             (None, 4)                 68        
_________________________________________________________________
activation_17 (Activation)   (None, 4)                 0         
_________________________________________________________________
dense_18 (Dense)             (None, 1)                 5         
_________________________________________________________________
activation_18 (Activation)   (None, 1)                 0         
Total params: 73
Trainable params: 73
Non-trainable params: 0
_________________________________________________________________
Train on 491 samples, validate on 123 samples
Epoch 1/1000
 - 1s - loss: 0.6293 - acc: 0.6701 - val_loss: 0.6328 - val_acc: 0.6748
Epoch 2/1000
 -

Epoch 85/1000
 - 0s - loss: 0.4706 - acc: 0.7984 - val_loss: 0.5280 - val_acc: 0.8049
Epoch 86/1000
 - 0s - loss: 0.4705 - acc: 0.7984 - val_loss: 0.5282 - val_acc: 0.8049
Epoch 87/1000
 - 0s - loss: 0.4706 - acc: 0.7984 - val_loss: 0.5287 - val_acc: 0.8049
Epoch 88/1000
 - 0s - loss: 0.4706 - acc: 0.7984 - val_loss: 0.5287 - val_acc: 0.8049
Epoch 89/1000
 - 0s - loss: 0.4704 - acc: 0.7984 - val_loss: 0.5284 - val_acc: 0.8049
Epoch 90/1000
 - 0s - loss: 0.4703 - acc: 0.7984 - val_loss: 0.5283 - val_acc: 0.8049
Epoch 91/1000
 - 0s - loss: 0.4702 - acc: 0.7984 - val_loss: 0.5283 - val_acc: 0.8049
Epoch 92/1000
 - 0s - loss: 0.4702 - acc: 0.7984 - val_loss: 0.5283 - val_acc: 0.8049
Epoch 93/1000
 - 0s - loss: 0.4702 - acc: 0.7984 - val_loss: 0.5286 - val_acc: 0.8049
Epoch 94/1000
 - 0s - loss: 0.4702 - acc: 0.7984 - val_loss: 0.5289 - val_acc: 0.8049
Epoch 95/1000
 - 0s - loss: 0.4701 - acc: 0.7984 - val_loss: 0.5290 - val_acc: 0.8049
Epoch 96/1000
 - 0s - loss: 0.4700 - acc: 0.7984 - val

In [16]:
keras.backend.clear_session()

In [46]:
threshold = 0.25
OUT2 = []
for i in range(len(OUT)):
    if OUT[i][0] > threshold:
        OUT2.append('1')
    else:
        OUT2.append('0')
print(OUT2)

['1', '1', '1', '1', '1', '1', '1', '0', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '0', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '0', '1', '1', '1', '1', '1', '1', '1', '0', '1', '1', '1', '1', '1', '0', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '0', '1', '0', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '0', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '0', '0', '1', '1', '1', '1', '0', '0', '1', '0', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '0', '1', '0', '1', '1', '1', '1', '0', '1', '1', '1', '1', '1', '0', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '0', '1', '1', '1', '1', '1', '1', '0', '0', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '0', '0', '1', '1', '1', '1', '1', '1',

In [47]:
ANS = pd.DataFrame({'Loan_ID':Test.index, 'Loan_Status':OUT2}).replace({'1':'Y', '0':'N'})

In [48]:
ANS.to_csv('.\data\MLP_keras_th_'+str(threshold)+'_L_'+str(input_size)+ '_L2_'+ str(L2) +'_V2.csv',index = 0)

In [11]:
Train

,Gender,Married,Education,Self_Employed,ApplicantIncome,CoapplicantIncome,LoanAmount,Loan_Amount_Term,Credit_History,Dependents_0,Dependents_1,Dependents_2,Dependents_3+,Property_Area_Rural,Property_Area_Semiurban,Property_Area_Urban
Loan_ID,,,,,,,,,,,,,,,,
LP001002,0.0,0.0,1.0,0.0,0.072210,0.000000,0.214286,0.750,1.0,1.0,0.0,0.0,0.0,0,0,1
LP001003,0.0,1.0,1.0,0.0,0.056580,0.036192,0.182857,0.750,1.0,0.0,1.0,0.0,0.0,1,0,0
LP001005,0.0,1.0,1.0,1.0,0.037037,0.000000,0.094286,0.750,1.0,1.0,0.0,0.0,0.0,0,0,1
LP001006,0.0,1.0,0.0,0.0,0.031889,0.056592,0.171429,0.750,1.0,1.0,0.0,0.0,0.0,0,0,1
LP001008,0.0,0.0,1.0,0.0,0.074074,0.000000,0.201429,0.750,1.0,1.0,0.0,0.0,0.0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
LP002978,1.0,0.0,1.0,0.0,0.035802,0.000000,0.101429,0.750,1.0,1.0,0.0,0.0,0.0,1,0,0
LP002979,0.0,1.0,1.0,0.0,0.050691,0.000000,0.057143,0.375,1.0,0.0,0.0,0.0,1.0,1,0,0
LP002983,0.0,1.0,1.0,0.0,0.099654,0.005760,0.361429,0.750,1.0,0.0,1.0,0.0,0.0,0,0,1
